In [6]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

In [7]:
class Caller(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        # share max for all weights?
        
        w = [0, 0]
        m = [0, 0]
        w[0] = self.model.layers[0].get_weights()
        m[0] = np.abs(w[0]).max()
        
        w[1] = self.model.layers[1].get_weights()
        m[1] = np.abs(w[1]).max()
        
        mlast = np.array(m).max()
        print('\t ', mlast, m[0], m[1])
        self.model.layers[0].set_weights(w[0] / m[0])
        self.model.layers[1].set_weights(w[1] / m[1])

In [9]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 6

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape, use_bias=False))
model.add(Conv2D(64, (3, 3), activation='relu', use_bias=False))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[Caller()])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/6
	  0.140505 0.140505 0.0837752
 9088/60000 [===>..........................] - ETA: 123s - loss: 1.8755 - acc: 0.5964	  1.00036 1.00036 1.00008


18432/60000 [========>.....................] - ETA: 81s - loss: 1.3080 - acc: 0.6845	  1.00026 0.999958 1.00026


27776/60000 [============>.................] - ETA: 56s - loss: 1.0664 - acc: 0.7317	  1.00018 1.00018 1.00005


37120/60000 [=================>............] - ETA: 38s - loss: 0.9334 - acc: 0.7600	  1.00006 0.999786 1.00006


46464/60000 [======================>.......] - ETA: 21s - loss: 0.8301 - acc: 0.7829	  0.999954 0.999885 0.999954


...

26624/60000 [============>.................] - ETA: 41s - loss: 0.0771 - acc: 0.9784	  1.00006 0.999912 1.00006


35968/60000 [================>.............] - ETA: 30s - loss: 0.0764 - acc: 0.9778	  1.00016 1.00016 1.00001


45312/60000 [=====================>........] - ETA: 19s - loss: 0.0771 - acc: 0.9776	  1.00079 1.00063 1.00079


54656/60000 [==========================>...] - ETA: 7s - loss: 0.0777 - acc: 0.9777	  1.00012 0.999798 1.00012


60000/60000 [==============================] - 81s - loss: 0.0782 - acc: 0.9775 - val_loss: 0.0421 - val_acc: 0.9868
Test loss: 0.0421416799439
Test accuracy: 0.9868


In [10]:
model.load_weights('mnist_nobias_last')

In [20]:
print(model.layers[-1].weights[0])

<tf.Variable 'dense_6/kernel:0' shape=(128, 10) dtype=float32_ref>


In [4]:
import numpy as np
w = [0, 0]
m = [0, 0]
w[0] = model.layers[0].get_weights()
m[0] = np.abs(w[0]).max()

w[1] = model.layers[1].get_weights()
m[1] = np.abs(w[1]).max()

mlast = np.array(m).max()
print('\t ', mlast, m[0], m[1])
self.model.layers[0].set_weights(w[0] / mlast)
self.model.layers[1].set_weights(w[1] / mlast)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()